# <center> **Coded Apertures: 2D Image Recostruction Test**<center>

**Libraries**

In [2]:
import unittest
import collections.abc as c

import numpy as np
from sympy import nextprime
from matplotlib.colors import ListedColormap
from scipy.signal import correlate

import plot_module as plot

np.set_printoptions(precision=2, suppress=False)

**URA/MURA Pattern Simulation**

In [ ]:
class Mask_Pattern:
    """
    Simulates a 2D (n x n) URA/MURA pattern and its decoding
    function from a folded 1D sequence for a coded-mask camera.
    """

    def __init__(self, n: int):
        
        self.n = n
    

    def ura_pattern(self) -> tuple[c.Sequence, c.Sequence]:
        """Generates a folded URA mask pattern and its decoder."""
        
        root = nextprime(self.n**2)
        flag = np.round(((root - 3)/4) - ((root - 3)/4)) == 0

        if not flag:
            raise ValueError(f"length {self.n} not in URA sequences.")
        
        a = np.zeros(root)
        r = (np.arange(root)**2) % root
        a[r] = 1

        g = 2*a - 1

        return tuple(map(lambda x: np.reshape(x[:self.n ** 2], (self.n, self.n)).astype(int), [a, g]))


    def mura_pattern(self) -> tuple[c.Sequence, c.Sequence]:
        """Generates a folded MURA mask pattern and its decoder."""
        
        root = nextprime(self.n**2)
        flag = np.round(((root - 1)/4) - ((root - 1)/4)) == 0

        if not flag:
            raise ValueError(f"length {self.n} not in MURA sequences.")
        
        a = np.zeros(root)
        r = (np.arange(root)**2) % root
        a[r] = 1

        g = 2*a - 1

        a[0], g[0] = 0, 1

        return tuple(map(lambda x: np.reshape(x[:self.n ** 2], (self.n, self.n)).astype(int), [a, g]))

In [57]:
n = 19
flag = False

A_ura, G_ura = Mask_Pattern(n).ura_pattern()
A_mura, G_mura = Mask_Pattern(n).mura_pattern()

if flag:
    plot.image_plot([A_ura, G_ura],
                ["URA Mask Pattern", "URA Mask Decoder"],
                cbarlabel=["Aperture", None],
                cbarcmap=[ListedColormap(["DodgerBlue", "DeepSkyBlue"]), ListedColormap(["DeepPink", "Orange"])])

    plot.image_plot([A_mura, G_mura],
                ["MURA Mask Pattern", "MURA Mask Decoder"],
                cbarlabel=["Aperture", None],
                cbarcmap=[ListedColormap(["DodgerBlue", "DeepSkyBlue"]), ListedColormap(["DeepPink", "Orange"])])

    plot.image_plot([correlate(A_ura, G_ura, mode='full'), correlate(A_ura, G_ura, mode='same')],
                ["A*G mode='full'", "A*G mode='same'"],
                cbarlabel=["Correlation Value", "Correlation Value"],
                cbarcmap=[None, None])

**Mask Interface**

In [ ]:
class Coded_Mask_Interface(Mask_Pattern):
    """Interface for the coded mask camera analysis."""
    
    def __init__(self,
                 n: int,
                 sky_image: c.Sequence,
                 pattern_type: str):
        
        self.n = n
        self.pattern_type = pattern_type

        self.mask, self.decoder = self.pattern(pattern_type)
        self.open_fraction = self.mask.sum()/n**2

        self.sky_image = sky_image
        self.detector_image = None
        self.reconstructed_sky = None
    

    def pattern(self, pattern_type: str) -> tuple[c.Sequence, c.Sequence]:
        """Generates the mask pattern."""

        if pattern_type not in ['ura', 'mura']:
            raise ValueError(f"Invalid pattern_type '{pattern_type}', must be 'ura' or 'mura'.")
        
        elif pattern_type == 'ura':
            return self.ura_pattern()
        
        elif pattern_type == 'mura':
            return self.mura_pattern()
    

    def psf(self) -> c.Sequence:
        """Returns the mask PSF."""

        return correlate(self.mask, self.decoder, mode='same')
    

    def encode(self, detector_background_rate: None | float = None) -> c.Sequence:
        """Returns the detector image from the simulated sky image."""
        
        self.detector_image = correlate(self.mask, self.sky_image, mode='same')

        if detector_background_rate:
            self.detector_image += np.random.poisson(detector_background_rate,
                                                     self.detector_image.shape)
        
        return self.detector_image


    def decode(self) -> c.Sequence:
        """Returns the reconstructed sky image from the detector image."""
        
        self.reconstructed_sky = correlate(self.decoder, self.detector_image, mode='same')

        return self.reconstructed_sky
    

    def snr(self) -> c.Sequence:
        """Returns the SNR for the reconstructed image."""

        return self.reconstructed_sky/np.sqrt(self.reconstructed_sky.sum())
    

    @property
    def mask_shape(self):
        return self.mask.shape
    
    @property
    def sky_image_shape(self):
        return self.sky_image.shape
    
    @property
    def decoder_shape(self):
        return self.decoder.shape
    
    @property
    def detector_image_shape(self):
        return self.detector_image.shape

In [ ]:
cai = Coded_Mask_Interface(19, np.array([0, 3, 0, 6]), 'mura')

cai.n, cai.pattern_type, cai.mask
cai.decoder, cai.open_fraction, cai.mask.shape
cai.psf(), cai.psf().shape
cai.mask_shape, cai.sky_image_shape

((19, 19), (4,))

**Sky Simulation**

In [76]:
def simulate_sky_image(n: int,
                       sources_flux: c.Sequence,
                       sources_pos: None | c.Sequence[tuple] = None,
                       sky_background_rate: None | float = None,
                       ) -> c.Sequence | tuple[c.Sequence, c.Sequence]:
    """Simulates the sky image given the sources flux."""

    sky_image = np.zeros((n, n))

    # assign fluxes to point-like sources
    if sources_pos is not None:
        for i, pos in enumerate(sources_pos):
            sky_image[pos[0], pos[1]] = sources_flux[i]
    
    else:
        for flux in sources_flux:
            x, y = np.random.randint(0, n + 1), np.random.randint(0, n + 1)
            sky_image[x, y] = flux
    
    # add sky background
    if sky_background_rate is not None:
        sky_background = np.random.poisson(sky_background_rate, (n, n))
        sky_image += sky_background

        return sky_image, sky_background
    
    else:    
        return sky_image

In [83]:
n = 40
sources_flux = [50, 30, 40]
sources_pos = [(20, 30), (10, 20), (30, 10)]
sky_background_rate=5

flag = False

S1 = simulate_sky_image(n, sources_flux)
S2 = simulate_sky_image(n, sources_flux, sources_pos)
S3, sky_B3 = simulate_sky_image(n, sources_flux, sources_pos, sky_background_rate)

if flag:
    plot.image_plot([S1, S2, S3, sky_B3],
                    ["S1", "S2", "S3", "sky_B3"],
                    cbarcmap=[None]*4)

**Sky Reconstruction Simulation**

In [ ]:
def simulation_routine():
    """Simulates the reconstruction of the sky for a coded mask camera."""
    pass




def simulate(
    flux_sources: list, 
    sky_background_rate: float, 
    mask: URA, 
    sources_pos: list[tuple[int, int]]=None
    ) -> tuple[detector_array, sky_true_array, dict('sources direction': source_pos, 'transmitted_source_fluxes': )]: 
    """Simulate a real sky image"""

    return 1

**Signal-to-Noise Ratio**

**Tests**

### **Random Mask Pattern**

### **URA Mask Pattern**